# Mixed models

This repository shows the different algorithmic approaches to fit generalized linear mixed models.

\begin{align}
\mathbf{y} \mid \mathbf{u} &\sim \mathcal{N}\left(\mathbf{X}\boldsymbol \beta + \mathbf{Z}\mathbf{u}, \sigma^2 \mathbf{I}\right)\\
\mathbf{u} & \sim  \mathcal{N}\left(\mathbf{0}, \mathbf{Q}\right)
\end{align}

\begin{align}
\mathbf{y} \sim \mathcal{N}\left(\mathbf{X}\boldsymbol \beta, \sigma^2 \mathbf{I} +   \mathbf{Z} \mathbf{Q}  \mathbf{Z}^T\right)
\end{align}

In [23]:
import statsmodels
from patsy import dmatrices, dmatrix, demo_data

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas
from sklearn.preprocessing import LabelEncoder

In [6]:
tab = pandas.read_csv("./data/sleepstudy.csv")

In [35]:
Y, X = dmatrices("Reaction ~ Days", tab)

In [44]:
def _build_ranef_model_matrix(tab, factor, ranef):
    inter_tab = tab[[factor, ranef]].copy()    
    inter_tab['grp'] = LabelEncoder().fit_transform(tab.Subject)
    inter_tab = inter_tab[[factor, "grp"]].pivot(columns=factor).reindex()
    inter_tab.values[np.isfinite(inter_tab.values)] = 1
    inter_tab.values[np.isnan(inter_tab.values)] = 0
    
    slope_tab = tab[[factor, ranef]].copy().pivot(columns=factor).reindex()
    slope_tab.values[np.isnan(slope_tab.values)] = 0
    
    Z = pd.concat([inter_tab, slope_tab], axis=1, sort=True)
    Z = Z.reindex(sorted(Z.columns, key=lambda x: x[1]), axis=1)
    
    return Z
Z = _build_ranef_model_matrix(tab, "Subject", "Days")

In [46]:
Z

,grp,Days,grp,Days,grp,Days,grp,Days,grp,Days,...,grp,Days,grp,Days,grp,Days,grp,Days,grp,Days
Subject,308,308,309,309,310,310,330,330,331,331,...,352,352,369,369,370,370,371,371,372,372
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
